# Code conversion using Gemini and Codestral in Windows 11

In [ ]:
import os
import io
import sys
import gradio as gr
import subprocess
from google import genai
from google.genai import types
from mistralai import Mistral
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display

In [ ]:
# Load Gemini and Mistral API Keys

load_dotenv(override=True)
gemini_api_key = os.getenv("GOOGLE_API_KEY")
mistral_api_key = os.getenv("MISTRAL_API_KEY")

if not mistral_api_key or not gemini_api_key:
    print("API Key not found!")
else:
    print("API Key loaded in memory")

In [ ]:
# Models to be used

MODEL_GEMINI = 'gemini-2.5-flash'
MODEL_CODESTRAL = 'codestral-latest'

In [ ]:
# Load Gemini client
try:
    gemini_client = genai.Client(api_key=gemini_api_key)
    print("Google GenAI Client initialized successfully!")

    codestral_client = Mistral(api_key=mistral_api_key)
    print("Mistral Client initialized successfully!")
except Exception as e:
    print(f"Error initializing Client: {e}")
    exit() 

In [ ]:
# Gemini System prompt

system_message = "You are an assistant that reimplements Python code in high-performance C++ optimized for a Windows PC. "
system_message += "Use Windows-specific optimizations where applicable (e.g., multithreading with std::thread, SIMD, or WinAPI if necessary). "
system_message += "Respond only with the equivalent C++ code; include comments only where absolutely necessary. "
system_message += "Avoid any explanation or text outside the code. "
system_message += "The C++ output must produce identical functionality with the fastest possible execution time on Windows."

generate_content_config = types.GenerateContentConfig(system_instruction=system_message)

In [ ]:
def user_prompt_for(python):
    user_prompt = (
        "Convert the following Python code into high-performance C++ optimized for Windows. "
        "Use standard C++20 or newer with Windows-compatible libraries and best practices. "
        "Ensure the implementation runs as fast as possible and produces identical output. "
        "Use appropriate numeric types to avoid overflow or precision loss. "
        "Avoid unnecessary abstraction; prefer direct computation and memory-efficient structures. "
        "Respond only with C++ code, include all required headers (like <iomanip>, <vector>, etc.), and limit comments to only what's essential.\n\n"
    )
    user_prompt += python
    return user_prompt

In [ ]:
def user_message_gemini(python): 
    return types.Content(role="user", parts=[types.Part.from_text(text=user_prompt_for(python))])  

In [ ]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [ ]:
def write_output(cpp):
    code = cpp.replace("```cpp", "").replace("```c++", "").replace("```", "").strip()
   
    if not "#include" in code:
        raise ValueError("C++ code appears invalid: missing #include directives.")

    with open("optimized.cpp", "w", encoding="utf-8", newline="\n") as f:
        f.write(code)  

In [ ]:
# Generate CPP code using Gemini

def optimize_gemini(python):
    stream = gemini_client.models.generate_content_stream(
        model = MODEL_GEMINI,
        config=generate_content_config,
        contents=user_message_gemini(python)
    )
    cpp_code = ""
    for chunk in stream:
        chunk_text = chunk.text
        cpp_code += chunk_text
        print(chunk_text, end="", flush=True)    
    write_output(cpp_code)

In [ ]:
# Generate CPP code using Codestral

def optimize_codestral(python):
    stream = codestral_client.chat.stream(
        model = MODEL_CODESTRAL,
        messages = messages_for(python),        
    )
    
    cpp_code = ""
    for chunk in stream:
        chunk_text = chunk.data.choices[0].delta.content
        cpp_code += chunk_text
        print(chunk_text, end="", flush=True)    
    write_output(cpp_code)

In [ ]:
# Actual code to convert

pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [ ]:
exec(pi)

In [ ]:
optimize_gemini(pi)

In [ ]:
# CPP Compilation

!g++ -O3 -std=c++20 -o optimized.exe optimized.cpp

In [ ]:
!.\optimized.exe

In [ ]:
optimize_codestral(pi)

In [ ]:
!g++ -O3 -std=c++20 -o optimized.exe optimized.cpp

In [ ]:
!.\optimized.exe

## Hard Code

In [ ]:
python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [ ]:
exec(python_hard)

In [ ]:
optimize_gemini(python_hard)

In [ ]:
# CPP Compilation

!g++ -O3 -std=c++20 -o optimized.exe optimized.cpp

In [ ]:
!.\optimized.exe

In [ ]:
optimize_codestral(python_hard)

In [ ]:
# CPP Compilation

!g++ -O3 -std=c++20 -o optimized.exe optimized.cpp

In [ ]:
!.\optimized.exe

## Accommodating the entire code in Gradio

In [ ]:
def stream_gemini(python):
    stream = gemini_client.models.generate_content_stream(
        model = MODEL_GEMINI,
        config=generate_content_config,
        contents=user_message_gemini(python)
    )

    cpp_code = ""
    for chunk in stream:
        chunk_text = chunk.text or ""
        cpp_code += chunk_text
        yield cpp_code.replace('```cpp\n','').replace('```','')

In [ ]:
def stream_codestral(python):
    stream = codestral_client.chat.stream(
        model = MODEL_CODESTRAL,
        messages = messages_for(python),        
    )

    cpp_code = ""
    for chunk in stream:
        chunk_text = chunk.data.choices[0].delta.content or ""
        cpp_code += chunk_text
        yield cpp_code.replace('```cpp\n','').replace('```','')    

In [ ]:
def optimize(python, model):
    if model.lower() == 'gemini':
        result = stream_gemini(python)
    elif model.lower() == 'codestral':
        result = stream_codestral(python)
    else:
        raise ValueError("Unknown model")
        
    for stream_so_far in result:
        yield stream_so_far        

### Gradio Implementation

In [ ]:
custom_css = """
.scrollable-box textarea {
    overflow: auto !important;
    height: 400px;
}

.python {background-color: #306998;}
.cpp {background-color: #050;}

"""

theme = gr.themes.Soft()

In [ ]:
def execute_python(code):
    try:
        result = subprocess.run(
            ["python", "-c", code],
            capture_output=True,
            text=True,
            timeout=60
        )
        if result.returncode == 0:
            return result.stdout or "[No output]"
        else:
            return f"[Error]\n{result.stderr}"
    except subprocess.TimeoutExpired:
        return "[Error] Execution timed out."
    except Exception as e:
        return f"[Exception] {str(e)}"  

In [ ]:
def execute_cpp(code):
    write_output(code)
    
    try:
        compile_cmd = ["g++", "-O3", "-std=c++20", "-o", "optimized.exe", "optimized.cpp"]
        compile_result = subprocess.run(compile_cmd, capture_output=True, text=True, check=True)
        
        run_cmd = ["optimized.exe"]
        run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True, timeout=60)
        
        return run_result.stdout or "[No output]"
        
    except subprocess.CalledProcessError as e:
        return f"[Compile/Runtime Error]\n{e.stderr}"
    except subprocess.TimeoutExpired:
        return "[Error] Execution timed out."
    except Exception as e:
        return f"[Exception] {str(e)}"  

In [ ]:
with gr.Blocks(css=custom_css, theme=theme) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=10, value=python_hard, elem_classes=["scrollable-box"])
        cpp = gr.Textbox(label="C++ code:", lines=10, elem_classes=["scrollable-box"])
    with gr.Row():
        model = gr.Dropdown(["Gemini", "Codestral"], label="Select model", value="Gemini")
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])

    convert.click(optimize, inputs=[python,model], outputs=[cpp])
    python_run.click(execute_python,inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)   